In [1]:
import pandas as pd
from tqdm import tqdm
from natsort import natsorted
import os, json, open_clip, torch
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

import os
import json
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from natsort import natsorted
import pandas as pd
from tqdm import tqdm
import open_clip
from datetime import datetime
from open_clip import create_model_from_pretrained, get_tokenizer

In [2]:
# 0. Settings
device = 'cuda:0'
model_name = 'ViT-L-14-336'
pretrained = 'openai'
root = './'
dataset_name = 'Scene'

# 1. Load CLIP model
model, preprocess = open_clip.create_model_from_pretrained(model_name, pretrained=pretrained)
tokenizer = open_clip.get_tokenizer(model_name)

# 2. Load test dataset with data augmentation
ds = ImageFolder(os.path.join(root, dataset_name), transform=preprocess)
ds.samples = natsorted(ds.samples)
dl = DataLoader(ds, shuffle=False, batch_size=32, num_workers=2)

# Generate detailed prompts

class_names= ['Buildings', 'Forests', 'Glacier', 'Mountains', 'Sea', 'Street']
labels_list = []

for class_name in class_names:
    labels_list.extend([
        f"A photo of a {class_name}",
        f"An image depicting a {class_name}",
        f"A scenic view of {class_name}",
        f"A {class_name} landscape",
        f"A snapshot of a {class_name}",
        f"A beautiful {class_name} scene",
        f"An artistic representation of {class_name}",
        f"A {class_name} captured in nature",
        f"A {class_name} during sunset",
        f"A serene {class_name} environment"
    ])
    
    labels_list.extend([
    f"A breathtaking view of {class_name}",
    f"A majestic {class_name} landscape",
    f"A vibrant {class_name} scene",
    f"A {class_name} in the early morning light",
    f"A dramatic depiction of {class_name}",
    f"A {class_name} immersed in fog",
    f"A serene and peaceful {class_name}",
    f"A rugged {class_name} terrain",
    f"A {class_name} teeming with life",
    f"A sun-drenched {class_name}"
    ])
    
    labels_list.extend([
    f"What a beautiful {class_name}!",
    f"This is a {class_name} scene.",
    f"Imagine a {class_name} like this.",
    f"Picture a serene {class_name}.",
    f"The essence of {class_name} is captured here.",
    f"Nature's {class_name} at its best.",
    f"A true depiction of {class_name}.",
    f"The tranquility of {class_name}.",
    f"The rugged beauty of {class_name}.",
    f"The timelessness of {class_name}."
    ])
    
    labels_list.extend([
    f"A {class_name} at sunrise",
    f"A rainy day in the {class_name}",
    f"A snowy {class_name} landscape",
    f"A bright sunny day over the {class_name}",
    f"A {class_name} covered in mist",
    f"An overcast {class_name} scene",
    f"A {class_name} during golden hour",
    f"A {class_name} under a clear blue sky",
    f"A windy day in the {class_name}",
    f"A {class_name} under a starry night"
    ])
    
    labels_list.extend([
    f"People hiking in the {class_name}",
    f"Birds flying over the {class_name}",
    f"Children playing near the {class_name}",
    f"A festival happening in the {class_name}",
    f"Boats sailing on the {class_name}",
    f"A picnic by the {class_name}",
    f"Wildlife roaming in the {class_name}",
    f"A campsite in the {class_name}",
    f"A storm approaching the {class_name}",
    f"A serene sunset at the {class_name}"
    ])
    
    labels_list.extend([
    f"A bird's-eye view of the {class_name}",
    f"A close-up shot of {class_name}",
    f"A panoramic view of {class_name}",
    f"A view from the top of a {class_name}",
    f"A distant shot of the {class_name}",
    f"A view of {class_name} through the trees",
    f"A side view of {class_name}",
    f"A low-angle shot of the {class_name}",
    f"A view from within the {class_name}",
    f"A wide-angle shot capturing the {class_name}"
    ])
    
    labels_list.extend([
    f"The solitude of the {class_name}",
    f"The awe-inspiring {class_name}",
    f"A melancholic {class_name} scene",
    f"A {class_name} filled with hope",
    f"The beauty of simplicity in the {class_name}",
    f"The grandeur of {class_name}",
    f"A nostalgic view of {class_name}",
    f"The peaceful calm of {class_name}",
    f"A {class_name} that inspires wonder",
    f"A scene that stirs the soul: {class_name}"
    ])
    
    labels_list.extend([
    f"A {class_name} bathed in golden sunlight",
    f"The intricate details of a {class_name}",
    f"Shadow patterns on a {class_name}",
    f"The vibrant colors of a {class_name}",
    f"A {class_name} contrasted against a bright sky",
    f"A {class_name} in soft morning light",
    f"Textures and layers of a {class_name}",
    f"A {class_name} with reflections in the water",
    f"Light filtering through a {class_name}",
    f"Rich shadows falling over the {class_name}"
    ])
    
    labels_list.extend([
    f"A {class_name} covered in fresh snow",
    f"A {class_name} during a spring bloom",
    f"Golden autumn leaves around a {class_name}",
    f"A {class_name} at dusk, under a purple sky",
    f"A {class_name} in the crisp air of winter",
    f"A {class_name} at dawn with dewdrops",
    f"A {class_name} under the summer sun",
    f"A {class_name} in the cool shade of evening",
    f"A {class_name} blanketed in fog on a winter morning",
    f"A {class_name} glowing under the full moon"
    ])
    
    labels_list.extend([
    f"The quiet majesty of a {class_name}",
    f"A {class_name} that whispers tales of old",
    f"The eternal beauty of the {class_name}",
    f"A {class_name} that sings the songs of the earth",
    f"A {class_name} veiled in the mist of time",
    f"The {class_name} where dreams meet reality",
    f"A {class_name} that holds the secrets of nature",
    f"A {class_name} painted by the hands of the universe",
    f"The {class_name}, a canvas of the soul",
    f"A {class_name} where the earth meets the sky"
    ])

prompts = labels_list

# Perform zero-shot classification
zero_shot_top1 = 0
submission = dict({'id_idx': list(range(len(ds))), 'label': []})

with torch.no_grad(), torch.cuda.amp.autocast():
    text = tokenizer(prompts)
    text_features = model.encode_text(text).to('cpu')
    text_features /= text_features.norm(dim=-1, keepdim=True)

model = model.to(device)

for x, y in tqdm(dl):
    x = x.cuda(device)
    with torch.no_grad(), torch.cuda.amp.autocast():
        image_features = model.encode_image(x).to('cpu').float()
        image_features /= image_features.norm(dim=-1, keepdim=True)
        zero_shot_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)
        zero_shot_pred = zero_shot_probs.argmax(dim=-1).tolist()
        # Map predictions to class indices 0-5
        zero_shot_pred_mapped = [pred // 100 for pred in zero_shot_pred]
        submission['label'] += zero_shot_pred_mapped

# 5. Save prediction as submission.scv file.
file_name = 'OPEN-P100.csv'
print(file_name)
pd.DataFrame(submission).to_csv(os.path.join(root+"submissions/", file_name), index=False)

100%|█████████████████████████████████████████| 254/254 [02:05<00:00,  2.02it/s]

OPEN-P100.csv
